In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import efficientnet.keras as efn 
import librosa
import librosa.display as display
import os
import matplotlib.pyplot as plt
import matplotlib
from PIL import Image
from sklearn.utils import class_weight
import warnings
from tqdm import tqdm
#from tensorflow.keras import backend

from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D
from kapre.augmentation import AdditiveNoise
from kapre.time_frequency import Spectrogram

import multiprocessing as mp

%matplotlib inline

#!rm -r train_data
#!rm -r val_data
#!rm -r models
#!mkdir models

# suppress warnings
warnings.filterwarnings("ignore")

# fix for RTX2080 CTRNN memory alloc issues
SOUND_DIR = "/project/data/birdsong-recognition/train_audio/"
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

Using TensorFlow backend.


In [2]:
# function for creating Mel Spectrogram
def createMelSpectrogram(input_path, fileName, output_path, saveOrShow=0):
    print("creating for",fileName)
    if (os.path.isdir(os.path.join(input_path, fileName))):
        return
    
    # load sound signal
    signal, sr = librosa.load(os.path.join(input_path, fileName), duration=10)
    
    # create Mel Spectrogram
    S = Melspectrogram(n_dft=1024, 
                       n_hop=256,
                       input_shape=(1, signal.shape[0]),
                       padding='same', sr=sr, n_mels=224, fmin=1400, fmax=sr/2,
                       power_melgram=2.0, return_decibel_melgram=True,
                       trainable_fb=False, trainable_kernel=False)(signal.reshape(1, 1, -1)).numpy()
    
    S = S.reshape(S.shape[1], S.shape[2])
    
    ##### NOTE CHANGE TO FILENAME INDEX TO CORRECTLY PROCESS AUGMENT
    ##### FILES THAT CONTAIN A. PREFIXES AND OTHER DOTS BEFORE ".mp3"
    if saveOrShow == 0:   
        print("writing out", os.path.join(output_path, fileName[:-4] + ".png"))
        matplotlib.image.imsave(os.path.join(output_path, fileName[:-4] + ".png"), S)
    else:
        #plt.imshow(S)
        #plt.show()
        display.specshow(S, sr=sr)
        plt.show()

## Visualizing Mel-Spectrogram of an audio

# bird: aldfly
createMelSpectrogram(SOUND_DIR+"aldfly", "XC134874.mp3", "", 1)
createMelSpectrogram(SOUND_DIR+"aldfly", "XC16967.mp3", "", 1)

# bird: ameavo
createMelSpectrogram(SOUND_DIR+"ameavo", "XC133080.mp3", "", 1)
createMelSpectrogram(SOUND_DIR+"ameavo", "XC139829.mp3", "", 1)

## Create Mel-Spectrogram for training

In [3]:
# select top 20 birds for training
BIRDS = os.listdir("data/birdsong-recognition/train_audio/")[0:20]

In [4]:
BIRDS

['amered',
 'yelwar',
 'semsan',
 'horlar',
 'vigswa',
 'blugrb1',
 'cedwax',
 'sposan',
 'brthum',
 'gockin',
 'belkin1',
 'olsfly',
 'whbnut',
 'commer',
 'wewpew',
 'moudov',
 'wilsni1',
 'pasfly',
 'gryfly',
 'annhum']

In [5]:
# list of birds
#BIRDS = ["aldfly", "ameavo", "amebit", "amecro", "amegfi",
#         "amekes", "amepip", "amered", "amerob", "amewig"]

train_folder = "data/train_data_decibel_20/"
val_folder = "data/val_data_decibel_20/"

if not os.path.exists(train_folder): os.mkdir(train_folder)
if not os.path.exists(val_folder): os.mkdir(val_folder)

In [6]:
def bird_to_mel(bird):
    print(bird, "\n")
    INPUT_DIR = os.path.join("data/birdsong-recognition/train_audio/", bird)
    TRAIN_DIR = os.path.join(train_folder, bird)
    VAL_DIR = os.path.join(val_folder, bird)
    
    # create folders
    if not(os.path.exists(TRAIN_DIR)) and not(os.path.exists(VAL_DIR)): 
        
        os.mkdir(TRAIN_DIR)
        os.mkdir(VAL_DIR)

        
        # split into train and val set
        for f in os.listdir(INPUT_DIR):
            
            rand = np.random.randint(0, 10)

            if rand <= 7: 
                createMelSpectrogram(INPUT_DIR, f, TRAIN_DIR)
            else:
                createMelSpectrogram(INPUT_DIR, f, VAL_DIR)        

In [7]:
### This takes long time to run ###

## Needed to manualy rename the train and validation data directories
## to force it to be recreated with the full data set

# create train and val spectrogram
np.random.seed(1234)

threads = int((mp.cpu_count() /2) - 1)
# threads = int(mp.cpu_count() -2)
# threads = 48

# Handle single-core machines
if (threads < 1): threads = 1
print(f"Launching {threads} threads")
pool = mp.Pool(threads)

pool.map(bird_to_mel, BIRDS)



Launching 31 threads
yelwarameredhorlarsemsancedwaxblugrb1sposanvigswabrthumbelkin1olsflygockincommer  whbnutwewpew  moudovwilsni1  gryflypasfly  annhum   
   
 
 
 

  
 


































[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [8]:
IM_SIZE = (224,224,3)
BATCH_SIZE = 10

In [9]:
train_datagen = ImageDataGenerator(preprocessing_function=None,
                                   rescale=1/255,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.1,
                                   fill_mode='nearest')

train_batches = train_datagen.flow_from_directory(train_folder,
                                                  classes=BIRDS, 
                                                  target_size=IM_SIZE[0:2], 
                                                  class_mode='categorical', 
                                                  shuffle=True, batch_size=BATCH_SIZE)

val_datagen = ImageDataGenerator(preprocessing_function=None, rescale=1/255)

val_batches = train_datagen.flow_from_directory(val_folder,
                                                  classes=BIRDS, 
                                                  target_size=IM_SIZE[0:2], 
                                                  class_mode='categorical', 
                                                  shuffle=False, batch_size=1)

class_weights = class_weight.compute_class_weight('balanced', 
                                                  np.unique(train_batches.classes), 
                                                  train_batches.classes)

Found 15133 images belonging to 20 classes.
Found 3867 images belonging to 20 classes.


In [10]:
net = efn.EfficientNetB3(include_top=False, weights="imagenet", input_tensor=None, input_shape=IM_SIZE)
#net.trainable = False

x = net.output

#x1 = tf.keras.layers.GlobalAveragePooling2D()(x)
#x2 = tf.keras.layers.GlobalMaxPool2D()(x)
#x = tf.keras.layers.Concatenate(axis=1)([x1, x2])

x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.5)(x)

output_layer = tf.keras.layers.Dense(len(BIRDS), activation='softmax', name='softmax')(x)
net_final = tf.keras.Model(inputs=net.input, outputs=output_layer)

net_final.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
#net_final.summary()

In [12]:
ModelCheck = tf.keras.callbacks.ModelCheckpoint('models/efficientnet_checkpoint.h5', monitor='val_loss', verbose=0, 
                             save_best_only=True, save_weights_only=True, mode='auto')

net_final.reset_states()
net_final.fit_generator(train_batches, 
                        validation_data=val_batches,
                        steps_per_epoch = int(len(train_batches.classes)/BATCH_SIZE)+1,
                        validation_steps=len(val_batches.classes),
                        epochs=60, 
                        callbacks=[ModelCheck],
                       class_weight={i:class_weights[i] for i in range(len(BIRDS))})

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/60
1514/1514 [==============================] - 246s 162ms/step - loss: 1.8413 - accuracy: 0.4542 - val_loss: 1.5681 - val_accuracy: 0.6033
Epoch 2/60
1514/1514 [==============================] - 242s 160ms/step - loss: 1.1271 - accuracy: 0.6658 - val_loss: 1.1338 - val_accuracy: 0.6897
Epoch 3/60
1514/1514 [==============================] - 247s 163ms/step - loss: 0.8762 - accuracy: 0.7406 - val_loss: 0.8772 - val_accuracy: 0.7623
Epoch 4/60
1514/1514 [==============================] - 245s 162ms/step - loss: 0.7076 - accuracy: 0.7938 - val_loss: 0.8177 - val_accuracy: 0.7848
Epoch 5/60
1514/1514 [==============================] - 242s 160ms/step - loss: 0.5891 - accuracy: 0.8246 - val_loss: 0.7148 - val_accuracy: 0.8477
Epoch 6/60
1514/1514 [==============================] - 244s 161ms/step - loss: 0.4981 - accuracy: 0.8552 - val_loss: 0.4084 - val_accuracy: 0.8893
Epoch 7/60
1514/1514 [==============

1514/1514 [==============================] - 241s 159ms/step - loss: 0.0599 - accuracy: 0.9813 - val_loss: 0.2252 - val_accuracy: 0.9591
Epoch 55/60
1514/1514 [==============================] - 241s 159ms/step - loss: 0.0695 - accuracy: 0.9784 - val_loss: 0.2614 - val_accuracy: 0.9483
Epoch 56/60
1514/1514 [==============================] - 243s 161ms/step - loss: 0.0665 - accuracy: 0.9805 - val_loss: 0.3311 - val_accuracy: 0.9364
Epoch 57/60
1514/1514 [==============================] - 245s 162ms/step - loss: 0.0721 - accuracy: 0.9788 - val_loss: 0.2411 - val_accuracy: 0.9519
Epoch 58/60
1514/1514 [==============================] - 242s 160ms/step - loss: 0.0593 - accuracy: 0.9820 - val_loss: 0.5825 - val_accuracy: 0.8823
Epoch 59/60
1514/1514 [==============================] - 240s 159ms/step - loss: 0.0608 - accuracy: 0.9810 - val_loss: 0.2060 - val_accuracy: 0.9622
Epoch 60/60
1514/1514 [==============================] - 241s 159ms/step - loss: 0.0712 - accuracy: 0.9798 - val_loss:

In [13]:
net_final.load_weights('models/efficientnet_checkpoint.h5')
print("Accuracy on val data")
net_final.evaluate(val_batches, steps=len(val_batches.classes))[1]

Accuracy on val data
3867/3867 [==============================] - 57s 15ms/step - loss: 0.1764 - accuracy: 0.9638


0.9637961983680725